In [1]:
import sys
sys.path.append('../..')

from utils.normalization import normalizeAuditdTable
from utils.misc import getNotebookPath
from utils.preprocessing import readAndFilterFile
from utils.constants import *
import os
import pandas as pd


In [2]:
SCRIPT_PATH = getNotebookPath()

# input
DATA_FOLDER = SCRIPT_PATH + "\\..\\..\\data\\auditd_msft_raw\\"
LIMIT = 10
files = os.listdir(DATA_FOLDER)[:LIMIT]

out = []
for file in files:
    ldf = readAndFilterFile(DATA_FOLDER+file, normalizedFields=AUDITD_FIELDS, filterDict={"rule.sidid": AUDITD_TYPES})
    out.append(ldf)
out = pd.concat(out)
out.fillna("(none)", inplace=True)

In [3]:
df = normalizeAuditdTable(out.copy())

In [6]:
def get_label_example(label, col = 'auditd.summary.object.primary'):
    s = df[df[col].str.contains(label, regex=False)].iloc[0]
    host = s.hostname
    pid = s['process.pid']
    print(df[(df['hostname'] == host) & (df['process.pid'] == pid)][col])
    print(out[(out['hostname'] == host) & (out['process.pid'] == pid)][col])
    print(df[(df['hostname'] == host) & (df['process.pid'] == pid)][col].iloc[0])
    print(out[(out['hostname'] == host) & (out['process.pid'] == pid)][col].iloc[0])

get_label_example('(prvIP)')

16610    (prvIP)
Name: auditd.summary.object.primary, dtype: object
16610    10.0.0.55
Name: auditd.summary.object.primary, dtype: object
(prvIP)
10.0.0.55


In [7]:
get_label_example( '(pubIP)')

29216    (pubIP)
Name: auditd.summary.object.primary, dtype: object
29216    20.60.223.98
Name: auditd.summary.object.primary, dtype: object
(pubIP)
20.60.223.98


In [8]:
get_label_example('(IPv6)')

17520    (pubIP)
17560    (pubIP)
17562    (pubIP)
17561     (IPv6)
10065    (pubIP)
9794     (pubIP)
Name: auditd.summary.object.primary, dtype: object
10065           20.150.14.4
17520           20.150.14.4
17560         52.167.109.72
17561    2603:1030:408:4::e
17562         52.167.109.72
9794            20.150.14.4
Name: auditd.summary.object.primary, dtype: object
(pubIP)
20.150.14.4


In [9]:
get_label_example('(lopIP)', col='process.title')

6647    curl -m 10 -f -s -S http://(lopIP):10248/healthz
Name: process.title, dtype: object
6647    curl -m 10 -f -s -S http://127.0.0.1:10248/hea...
Name: process.title, dtype: object
curl -m 10 -f -s -S http://(lopIP):10248/healthz
curl -m 10 -f -s -S http://127.0.0.1:10248/healthz


In [10]:
get_label_example('(prvIP)', col='process.title')

19330    sh -c /opt/kuberner/bin/inject (pubIP):31337 (...
Name: process.title, dtype: object
19330    sh -c /opt/kuberner/bin/inject 1.2.3.4:31337 1...
Name: process.title, dtype: object
sh -c /opt/kuberner/bin/inject (pubIP):31337 (prvIP):22
sh -c /opt/kuberner/bin/inject 1.2.3.4:31337 10.42.148.130:22


In [11]:
label = '(prvIP)'
col='process.title'

#s = df[df[col].str.contains(label, regex=False)].iloc[0]
s = out[out[col].str.contains("/kuberner/bin/inject", regex=False)].iloc[0]
host = s.hostname
pid = s['process.pid']
print(df[(df['hostname'] == host) & (df['process.pid'] == pid)][col])
print(out[(out['hostname'] == host) & (out['process.pid'] == pid)][col])
print(df[(df['hostname'] == host) & (df['process.pid'] == pid)][col].iloc[0])
print(out[(out['hostname'] == host) & (out['process.pid'] == pid)][col].iloc[0])

19330    sh -c /opt/kuberner/bin/inject (pubIP):31337 (...
Name: process.title, dtype: object
19330    sh -c /opt/kuberner/bin/inject 1.2.3.4:31337 1...
Name: process.title, dtype: object
sh -c /opt/kuberner/bin/inject (pubIP):31337 (prvIP):22
sh -c /opt/kuberner/bin/inject 1.2.3.4:31337 10.42.148.130:22


In [12]:
get_label_example('(domain)', col='process.title')

257    /csi-node-driver-registrar --kubelet-registrat...
Name: process.title, dtype: object
257    /csi-node-driver-registrar --kubelet-registrat...
Name: process.title, dtype: object
/csi-node-driver-registrar --kubelet-registration-path=/var/lib/kubelet/plugins/(domain)/csi.sock --mode=kubelet-registration-probe
/csi-node-driver-registrar --kubelet-registration-path=/var/lib/kubelet/plugins/file.csi.azure.com/csi.sock --mode=kubelet-regis
